<a href="https://colab.research.google.com/github/honoreade/DeepLearning/blob/main/ImageDataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import os
import glob

# Paths
input_dir = "/content/drive/MyDrive/Colab Notebooks/Beauty/Moderate"  # Change to your image folder
output_dir = "/content/drive/MyDrive/Colab Notebooks/Beauty/LessBeauty"  # Where to save augmented images
os.makedirs(output_dir, exist_ok=True)

# Augmentation functions
def augment_image(image):
    """Apply a set of random augmentations to an image."""
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)

    # Random rotation (0, 90, 180, 270 degrees)
    image = tf.image.rot90(image, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

    return image

# Load and process images
image_paths = glob.glob(os.path.join(input_dir, "*.*"))  # Handles JPG and PNG

for img_path in image_paths:
    # Load image
    image = tf.io.read_file(img_path)

    # Decode based on file type
    if img_path.lower().endswith('.jpg') or img_path.lower().endswith('.jpeg'):
        image = tf.image.decode_jpeg(image, channels=3)
    elif img_path.lower().endswith('.png'):
        image = tf.image.decode_png(image, channels=3)
    else:
        continue  # Skip unsupported formats

    # Resize and normalize image
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0  # Normalize

    base_name = os.path.basename(img_path).split('.')[0]

    # Generate 4 augmentations per image
    for i in range(4):
        aug_image = augment_image(image)
        aug_image = tf.clip_by_value(aug_image, 0.0, 1.0)  # Ensure values are valid
        aug_image = tf.image.convert_image_dtype(aug_image, dtype=tf.uint8)  # Convert back to uint8

        # Encode and save
        encoded_image = tf.io.encode_jpeg(aug_image)
        new_filename = os.path.join(output_dir, f"{base_name}_aug_{i}.jpg")
        tf.io.write_file(new_filename, encoded_image)

print("Augmented images saved successfully!")

Augmented images saved successfully!
